In [ ]:
## This notebook provide functions to run RNAplex for structure prediction and vaRNA for 2nd structure plot

In [10]:
import os
RNAplex = '/home/beiliu/anaconda3/bin/RNAplex'
from IPython.display import Image as img
from Bio.Seq import Seq
import random

In [11]:
# RNAplex structure prediction
# dir for RNAplex prediction
os.system('mkdir RNAplex')
def get_RNA_plex_output(seq1, seq2, l = 40, c = 10, return_energy = False):
    '''
    function to run plex and get output
    seq1 and seq2 are the raw sequences
    seq2 will be the sequence comes first in the output seq_total
    l is the Maximal length of an interaction 
    c is Cost to add to each nucleotide in a duplex
    energy: return energy or not, default is False
    '''
    with open('RNAplex/seq1.fa', 'w+') as f:
        f.write('>seq1\n')
        f.write(str(seq1))
    with open('RNAplex/seq2.fa', 'w+') as f:
        f.write('>seq2\n')
        f.write(str(seq2))

    # run RNAplex
    os.system(f'{RNAplex} -q RNAplex/seq2.fa -t RNAplex/seq1.fa \
    -l {l} -c {c} > RNAplex/predict_result2.txt')

    # process RNAplex output and get duplex sequence and dbn
    with open('RNAplex/predict_result2.txt', 'r') as f:
        x = f.readlines()[-1]
#     print(x)
    # no reasonable predicted structure
    if x[0] == '>':
        return [0, 0, 0, 0, '..', 'AA', 0]
    else:
        try:
            dbn, pos_l, _, pos_r, energy, _, _ = x.split()
        except:
            dbn, pos_l, _, pos_r = x.split()[:4]
            energy = '(0)'

        [pos1_l, pos1_r] = [int(i) for i in pos_l.split(',')]
        [pos2_l, pos2_r] = [int(i) for i in pos_r.split(',')]

        seq_l = seq1[pos1_l-1:pos1_r]
        seq_R = seq2[pos2_l-1:pos2_r]
        seq_total = seq_l + '&' + seq_R
        
    if return_energy:
        energy = float(energy[1:-1])
        return pos1_l, pos1_r, pos2_l, pos2_r, dbn, seq_total, energy
    else:
        return pos1_l, pos1_r, pos2_l, pos2_r, dbn, seq_total

mkdir: cannot create directory ‘RNAplex’: File exists


In [4]:
# below is the function for detailed highlights

# vaRNA
# process DG clustering output and draw 2nd structures
vaRNA_script = '''#!/bin/bash \n \n\
java -cp /home/beiliu/Downloads/vaRNA/VARNAv3-93.jar \
fr.orsay.lri.varna.applications.VARNAcmd \
-sequenceDBN "{}" \
-structureDBN "{}" \
-algorithm radiate \
-baseInner "#334455" \
-baseName "#334455" \
-baseOutline "#334455" \
-bp "#334455" \
-bpStyle "line" \
-flat true -resolution "{}"  \
-highlightRegion "{}" \
-rotation "{}" -title "{}" \
-titleSize 10 -o {}.png
'''
random.seed = 1
def get_highlight_details(coor_list, color_list_input = 'random'):
    '''
    generate highlight regions and colors for vaRNA
    coor_list is a list of list (coordinates, start and end posistion)
    color list is the color of list, e.g. #FF0000, or it can be randomly generated
    '''
    if color_list_input == 'random':
        length = len(coor_list)
        r = lambda: random.randint(0,255)
        color_list = ['#%02X%02X%02X' % (r(),r(),r()) for i in range(length)]
    else:
        color_list = color_list_input
    highlightRegion_list = []
    for i, j in zip(coor_list, color_list):
        highlightRegion_list.append(f'{i[0]}-{i[1]}:fill={j}')
    if len(highlightRegion_list) == 1:
        return highlightRegion_list[0]
    else:
        return (';').join(highlightRegion_list)
    

# function to show 2nd structure image
def show_structure(dbn, seq, name1, name2, rotation = 20, suffix = '', res=2, arms = '',
                  coor_list = [[0,0]], color_list_input = 'random'):
    '''
    dbn, seq is the structure prediction results from get_RNA_plex_output (dbn and the corresponding RNA sequences)
    name1, name2 and arms are the names of the 2 strands plus the name of the interaction
    rotation is the layout of the output vaRNA figure
    suffix is the output path follow the directory /home/beiliu/scratch/KARR_seq/Tong_duplex_pred/vaRNA/vaRNA_output
    res is the resolution
    coor_list, color_list are the input of function get_highlight_details, a list of list (coordinates) and a list of colors
    e.g. #FF0000
    '''
    highlight = get_highlight_details(coor_list, color_list_input)
    with open(f'vaRNA/run_vaRNA.sh', 'w+') as f:
        f.write(vaRNA_script.format(seq, dbn, res,
                                    highlight,rotation,
                                    f'{name1}_{name2}_{arms}',
                                    f'vaRNA/vaRNA_output{suffix}'))
    os.system('bash vaRNA/run_vaRNA.sh')

In [9]:
print('''
usage: run get_RNA_plex_output(seq1, seq2, l = 40, c = 10) to get pos1_l, pos1_r, pos2_l, pos2_r, dbn, seq_total
then run show_structure()

e.g.:
pos1_l, pos1_r, pos2_l, pos2_r, dbn, seq_total \
= get_RNA_plex_output(seq1, seq2, c = 10, l=40)

show_structure(structure, seq1.replace('T', 'U'), f'motif{i+1}', motif, rotation = 0, 
               suffix = 'dummy', arms = seq_[1], res=5, highlight=(pos2_l, pos2_r))
prefix = 'vaRNA/'
img(f'{prefix}vaRNA_outputdummy.png')

''')


usage: run get_RNA_plex_output(seq1, seq2, l = 40, c = 10) to get pos1_l, pos1_r, pos2_l, pos2_r, dbn, seq_total
then run show_structure()

e.g.:
pos1_l, pos1_r, pos2_l, pos2_r, dbn, seq_total = get_RNA_plex_output(seq1, seq2, c = 10, l=40)

show_structure(structure, seq1.replace('T', 'U'), f'motif{i+1}', motif, rotation = 0, 
               suffix = 'dummy', arms = seq_[1], res=5, highlight=(pos2_l, pos2_r))
prefix = 'vaRNA/'
img(f'{prefix}vaRNA_outputdummy.png')




In [7]:
# SNORA73A sequence
SNORA73A_seq = \
'UCCAACGUGGAUACACCCGGGAGGUCACUCUCCCCGGGCUCUGUCCAAGUGGCGUAGGGGAGCAUAGGGCUCUGCCCCAUGAUGUACAAGUCCCUUUCCACAACGUUGGAAAUAAAGCUGGGCCUCGUGUCUGCGCCUGCAUAUUCCUACAGCUUCCCAGAGUCCUGUCGACAAUUACUGGGGAGACAAACCAUGCAGGAAACAGCC'

# SNORA73B sequence
SNORA73B_seq = \
'CCAACGUGGAUACCCUGGGAGGUCACUCUCCCCAGGCUCUGUCCAAGUGGCAUAGGGGAGCUUAGGGCUCUGCCCCAUGAUGUACAGUCCCUUUCCACAACGUUGAAGAUGAAGCUGGGCCUCGUGUCUGCGCCUGCAUAUUCCUACAGCUUCCCAGAGUCCUGUGGACAAUGACUGGGGAGACAAACCAUGCAGGAAACAUAU'


# SNORA73A dbn
SNORA73A_dbn = '.((((((((((((..(((.((((....))))....)))...)))))..((((.....(((((.(((((..........))..))).)...))))...)))).)))))))..........................(((((((..........((((((...((((...)))).......)))))).......)))))))........'

# SNORA73B dbn
SNORA73B_dbn = '((((((((((((.(((.((((....))))....)))...)))))..((((.....(((((.(((((..........))..))).)..))))...)))).)))))))..........................(((((((..........((((((...((((...)))).......)))))).......)))))))........'

In [8]:
show_structure(SNORA73B_dbn, SNORA73B_seq.replace('T', 'U'), f'SNORA73B', '', rotation = 0, 
               suffix = 'SNORA73B', arms = '', res=50, coor_list = [[114,123], [139, 150], [182, 190], [14,27]], color_list_input = ['#dea9b9', '#aacce3', '#aacce3', '#decc95'])
prefix = 'vaRNA/'
img(f'{prefix}vaRNA_outputSNORA73B.png')

Output file: /home/beiliu/scratch/KARR_seq/Tong_duplex_pred/vaRNA/vaRNA_outputSNORA73B.png

